**Context**: explaining machine learning algorithms is a broad topic and many methods have already been introduced, such as the popular [LIME](https://arxiv.org/pdf/1602.04938.pdf). However, explaining outliers remains especially difficult. This is because most methods explain a decision in creating an neighborhood around the instance to explain. But creating a neighborhood around an outlier can be difficult; by definition, the outlier is an isolated instance and its closest neighbor can be far away.

**Objective**: this notebook is an attempt to replicate the LIME method and adapt it in case of outliers.

**Method**: the method we used is structured as such:

1. Perform SMOTE to have a more balanced dataset

2. Define a neighborhood according to class ratio

3. Do a cross validation to find the best regularization strength

    a) Compute weights to give less importance on the furthest point
    
    b) Perform logistic regression
    
    
4. Explain using coefficients

# 1. SMOTE

[SMOTE](https://arxiv.org/pdf/1106.1813.pdf) is an algorithm allowing to oversample data that are imbalanced.

SMOTE algorithm works as such:

- loops on every outlier

- defines a neighborhood for each of them

- selects a random neighbor

- loops on every attribute and adds a random-weighted distance to the random neighbor: 

```generated_sample[attr] = outlier[attr] + (neighbor[attr]-outlier[attr])*random(0,1)```

In [ ]:
def performSMOTE(data, labels):
    oversample = SMOTE(sampling_strategy=0.05) # we want at least 5% of minority class
    data_aug, label_aug = oversample.fit_resample(data, labels)
    label_aug.columns = ['Outlier_Inlier']
    nb_generated_outliers = data_aug.shape[0] - data.shape[0]
    print("number of generated outliers: {}".format(nb_generated_outliers))
    return data_aug, label_aug

# 2. Define a neighborhood according to class ratio

In order to perform an efficient local classification, we need to have a relatively balanced neighoborhood. Thus, we want at least 10% of the neighborhood minor class.

We start with 20 neighbors and increase this number until we have a balanced neighborhood.

In [ ]:
def selectNeighborhood(data, labels, outlier):
    ratio_outliers = 1
    n_neighbors = 20
    while (ratio_outliers < 0.1 or ratio_outliers > 0.9): # we want at least 10% of the neighbor minor class
        n_neighbors = n_neighbors + 1
        if (n_neighbors % 100) == 0:
            print("trying with {} neighbors".format(n_neighbors))
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(data)
        distances, indices = nbrs.kneighbors(np.reshape(np.array(outlier),(1,-1)))
        distances = distances.flatten()
        indices = indices.flatten()
        X = data.iloc[indices,:]
        Y = labels.iloc[indices,:]
        nb_outliers_neighborhood = Y[Y['Outlier_Inlier']=='Outlier'].shape[0]
        nb_inliers_neighborhood = Y[Y['Outlier_Inlier']=='Inlier'].shape[0]
        try:
            ratio_outliers = nb_outliers_neighborhood/nb_inliers_neighborhood
        except:
            continue # in case of a division by zero
    print("{}% outliers in neighborhood using {} neighbors".format(int(ratio_outliers*100), n_neighbors))    
    return X, Y, indices, distances

# 3. Cross validation to find the best regularization strength

In order to have feature selection, we will classify the neighborhood using a lasso-regularized regression:

$$\underset{\theta}{\operatorname{argmin}}||Y - X\theta||^2 + \lambda|\theta|$$

In this section, $\lambda$ is found using cross validation: we perform multiple logistic regressions with different regularization constant. We look at the Average Precision Recall to assess the quality of the regression. The logistic regression uses weights that will be taken into account in the loss function as shown in the equation optimization:

$$\xi(x_0) = \underset{g \in G}{\operatorname{argmin}} ~\mathcal{L} (f, g, \pi_{x_0}) + \Omega(g)$$

As done in LIME, we use an exponential smoothing kernel to compute weights:

$$K(x,y) = e^{\frac{||x-y||^2_2}{\sigma^2}}$$

In [1]:
def kernel_rbf(x, y, sigma):
    d = 0
    for i in range(len(x)):
        d_temp = (x[i]-y[i])**2 
        d = d + d_temp
    return np.exp(-0.5*(d/sigma**2))

```sigma``` is the bandwidth representing the neighborhood size. LIME uses the following calculation: $bandwidth = 0.75*\sqrt{n_{features}}$. This seems quite arbitrary, and no documentation is provided regarding this formula.

It made more sense in our view to make the bandwidth dependent on the distance to the closest inlier in the neighborhood: $bandwidth = 0.05*\sqrt{d(Inlier)}$

In [ ]:
def computeBandwidth(labels, indices, distances):
    for idx, indice_neighbor in enumerate(indices):
        row = labels[labels['Outlier_Inlier'].index==indice_neighbor]
        if(row['Outlier_Inlier'].iloc[0]=='Inlier'):
            bandwidth = 0.05*np.sqrt(distances[idx])
            print("Bandwidth={}".format(bandwidth))
            break
    return bandwidth

We use ```StratifiedKFold``` in order to make sure we have balanced folds, meaning that we want at least one sample from each class in every fold.

We use Average Precision Recall score to compare the results. The Precision-Recall curve is better adapted than the ROC curve because

In [ ]:
def findBestRegularization(X, Y, outlier, bandwidth):
    n_splits = 3
    ave_pr_dict = {}
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    
    for lambda_reg in np.power(10,np.arange(1,10))/(10e4):
        
        ave_pr_mean = 0
        
        for train_index, test_index in kf.split(X, Y):
        
            X_train, X_test = np.array(X)[train_index], np.array(X)[test_index]
            Y_train, Y_test = np.array(Y)[train_index], np.array(Y)[test_index]
        
            # a) Compute weights with kernel. Inspired by LIME method https://arxiv.org/abs/1602.04938.
            outlier_list = np.reshape(np.array(outlier),(1,-1)) * np.ones((X_train.shape[0],X_train.shape[1]))
            sample_weight = []
            for idx, elt in enumerate(np.array(X_train)):
                sample_weight.append(kernel_rbf(elt, outlier_list[idx], bandwidth))
        
            # b) Perform logistic regression
            clf = LogisticRegression(random_state=42, solver='liblinear', penalty='l1', C=1/lambda_reg)
            clf.fit(X_train, np.array(Y_train).ravel(), sample_weight=sample_weight)
        
            proba = clf.predict_proba(X_test)
            
            ave_pr_mean = average_precision_score(Y_test, proba[:,1], pos_label='Outlier') + ave_pr_mean
    
        ave_pr_dict[lambda_reg] = ave_pr_mean / n_splits
    
    lambda_opt = max(ave_pr_dict, key=ave_pr_dict. get)
    print("best regularization strength: {}".format(lambda_opt))
    print("Average PR: {}".format(ave_pr_dict[lambda_opt]))
    return lambda_opt